This notebook explores a couple of the concepts we are pursuing for the next generation of the National Digital Catalog by using the USGS Core Research Center (CRC) as an example use case. The CRC manages collections of rock cores and cuttings at the USGS facility in Lakewood. The CRC has had a databased inventory for decades, the "CRC Well Catalog," that has been through several different generations of technology. This situation is generally typical across many of the other institutions managing geoscience collections (or other scientific collections for that matter). The CRC inventory management system consists of the following major technical components:

* A highly normalized relational database in PostgreSQL
* A custom web application for managing the database, accessed internally by CRC staff
* An ArcGIS MapServer [service](https://my.usgs.gov/arcgis/rest/services/crcwc/crcwc/MapServer) that provides feature services for the cores and cuttings collections along with supporting layers
* A custom [web application](https://my.usgs.gov/crcwc/) for public search and browse

One of the principles that we are pursuing for the next gen catalog is to try and make maximum use of whatever exists as operational infrastructure from NDC contributors. We want to move from a "push to" to a "draw from" modality. In the draw from mode, we are talking about software that is able to read whatever is available online and integrate the information into the NDC. So, the fundamental question is, how well does the CRC present its inventory to the world in a way that a software algorithm can read it and digest the information for use. We are examining this in terms of two main categories of data integration problem solving: syntactic and semantic.

So far, the CRC has only been represented in the NDC with a one-time data dump to CSV files of a small subset of its inventory. These records were organized into two collections in ScienceBase ([Cuttings](https://www.sciencebase.gov/catalog/item/4f4e49d8e4b07f02db5df2d2) and [Cores](https://www.sciencebase.gov/catalog/item/4f4e49dae4b07f02db5e0486)). In the process I lay out here, I removed the existing items and then replaced them with a new structure for the full set of items.

All in all, this process does work, and I used it to build out 65K items or so of new information in the NDC. However, there are a number of pretty major pitfalls encountered here that I discuss in the notes below.

In [1]:
from sciencebasepy import SbSession
from IPython.display import display
import os
import pandas as pd

To actually put items in ScienceBase, you need to be logged in. Since I am both deleting old items and inserting new items in the code below, I login with my credentials. NDC contributors who have access credentials of their own and one or more collections where write permissions have been granted could do something similar. If we are to run any "bots" like this in production, we will need to think about some type of service account(s) that have access permissions to build out various parts of the catalog.

In [2]:
sb = SbSession()
username = input("Username:  ")
sb.loginc(str(username))

Username:  sbristol@usgs.gov
········


This codeblock ran through the two collections (cores and cuttings) and deleted all of the existing child items imported under an older process. These items were severely out of date with the current live inventory, having been put into the system in 2011. Note: this is actually kind of pseudo-code as the ScienceBase API is not stable enough to support running through this entire deletion process without being restarted.

In [ ]:
for parent in ["4f4e49d8e4b07f02db5df2d2","4f4e49dae4b07f02db5e0486"]:
    items = sb.find_items({"max": 1000, "parentId": parent})
    while items and 'items' in items:
        for item in items['items']:
            sb.delete_item(item)
        items = sb.next(items)

I wrote all of the basic logic needed to make this system work into a set of functions called from a companion file here in this repo. I called it crcwc_to_sb.py, deliberately, because the majority of the logic contained in the file is very specific to this particular use case and will not really be functional anywhere else other than serving as a design pattern for a similar specialized case. Some aspects of the code, such as querying an ArcGIS MapServer service may be extensible beyond this specialized case.

In [3]:
from crcwc_to_sb import *

# Get data from ArcGIS MapServer
The first thing we do here is go out and retrieve all the records from the two MapServer layers that provide the basic "raw" records from the CRC Well Catalog. These two service layers were really set up for the purpose of driving the web map tool on the CRC web site. This is a generally common thing where a web service of some type was set up to drive a web tool, but it was not really intended as a data distribution service. This means that the services may provide reasonably good geometry for viewing points or other features on a map and some set of properties deemed appropriate for a "getFeatureInfo popup box," but are not necessarily the properties we would want to pull together into some other system.

In particular, the naming of the properties and the structure of the values may not be semantically conducive to integration with data from other systems. What seems to happen a lot of times is that folks will set up a web search app like the CRC has and then build some type of translational logic into the web app code that displays more human friendly property names. This is a problem when we want to write software to work with the information and take it off into some other context. That translational logic (part of the semantics) isn't encoded in any accessible form that the software can consult.

In the case of the CRC, the two MapServer layers are backed by views in the PostGIS database that assemble the set of properties that the designers decided they wanted to include. Not all of those properties are expressed through the web app, and the main manager for the CRC inventory was actually not even aware that the MapServer made them available at all (likely not an uncommon situation). Because it is driven by a view, meaning there is a configuration principal that sits between the data and its online API expression, we have the opportunity to go in and tweak it to potentially modify the field names and contents that are expressed to align with some standard.

There are about 65K records between these two layers, which takes a while to bring back from ArcGIS Server at its max of 1000 records per query, so I go ahead and cache two files locally for use.

In [4]:
%%time
required_files = ["core.json","cutting.json"]

if required_files == [f for f in required_files if os.path.isfile(f)]:
    with open('core.json', 'r') as f:
        cores = json.load(f)
        f.close()

    with open('cutting.json', 'r') as f:
        cuttings = json.load(f)
        f.close()
        
    df_cores = pd.DataFrame(cores["properties"])
    df_cuttings = pd.DataFrame(cuttings["properties"])

else:
    for sample_type in ["core","cutting"]:
        offset = 0
        crc_records = list()
        server_response = crcwc_items(sample_type=sample_type, offset=offset)
        while len(server_response["features"]) > 0:
            crc_records.extend(server_response["features"])
            offset += len(server_response["features"])
            server_response = crcwc_items(sample_type=sample_type, offset=offset)
        with open(f'{sample_type}.json', 'w') as f:
            json.dump(crc_records, f)

CPU times: user 837 ms, sys: 133 ms, total: 971 ms
Wall time: 1.02 s


# Mapping the Data
Figuring out how to map the CRC items from the two MapServer layers to ScienceBase took a little exploration through the data. I focused on meaningful identifiers along with the properties that I used to create titles and contacts. Everything else will have to be teased out at a later time.

As you can see in the next codeblocks, the property names and the contents are quite specific to the CRC and not easily translatable to some higher level standards or conventions. The information in this database is reflective of its long history. It started life in a FoxPro database (or possibly something older still that is now lost to history) and has been through various translations since without any significant work on reorganizing. This is really because of the core use of the database, which is for expert collection managers who know their information well to manage the physical plant itself. Select information from the collections are made available online for a specialized user community, but that information was never intended for the masses.

I think this is actually a very fundamental question that we should ask ourselves as a community as we head toward a new structure and design for the next gen catalog - just how much do we want to invest in harmonizing information across the catalog and moving it from a purely specialized audience to something at least somewhat broader.

I shoved the properties data from the two MapServer layers into Pandas dataframes for ease of use. The following tests run through the miniomal things I needed to know about the data in order to determine how to map into the ScienceBase Items. I include it here for future reference and to share a little bit about what's in these data for others to think about in different use cases.

## Common and Uncommon Properties
In this first codeblock, I take a look at the properties in each collection and compare them to see what's in common. Visually and content wise, it looks like there should be some additional properties in common that are just labeled a little differently in each collection for whatever historic reason. At the stage of expressing non-standardized and semantically aligned data to the NDC from various providers, it might be a good idea to at least harmonize across collections from the same provider source wherever possible to reduce the confusion at least a little bit.

In [ ]:
cores_properties = list(df_cores.columns)
cores_properties.sort()
cuttings_properties = list(df_cuttings.columns)
cuttings_properties.sort()
common_properties = list(set(cuttings_properties).intersection(set(cores_properties)))
common_properties.sort()

print("Properties in Cores")
print(cores_properties)
print("---------------")
print("Properties in Cuttings")
print(cuttings_properties)
print("---------------")
print("Properties in Common")
print(common_properties)


## Identifiers and Contacts
I'm going to try to pull any meaningful identifiers and contacts from the properties as possible as these are two of the available data structures in ScienceBase that we can leverage. In the following codeblock, I take a look at select properties from each collection that seem to have the information I want to use and show how many nulls are in each so I know how to write my item building code.

In [ ]:
print("Primary identifiers in Cores")
print("id", len(df_cores.loc[df_cores["id"].isnull()]))
print("libno", len(df_cores.loc[df_cores["libno"].isnull()]))
print("chlibno", len(df_cores.loc[df_cores["chlibno"].isnull()]))
print("apiwel", len(df_cores.loc[df_cores["apiwel"].isnull()]))
print("oper", len(df_cores.loc[df_cores["oper"].isnull()]))
print("====================")
print("Primary identifiers in Cuttings")
print("id", len(df_cuttings.loc[df_cuttings["id"].isnull()]))
print("chlibno", len(df_cuttings.loc[df_cuttings["chlibno"].isnull()]))
print("apinum", len(df_cuttings.loc[df_cuttings["apinum"].isnull()]))
print("operator", len(df_cuttings.loc[df_cuttings["operator"].isnull()]))

## Operator Contacts
The operator field in each collection contains a string name for what seems to usually be the drilling company that drilled the core or provided the cuttings. It seems to be reasonably populated, it could provide useful information linking to other systems, and there is a reasonable part of the ScienceBase data model to support the information that is added to the index as a search facet. However, when you look at the content, it's pretty messy and so of kind of limited use outside its original context. The following codeblock shows the top 100 records grouped on operator name from the cores collection to demonstrate this dynamic.

In [ ]:
for item in [(operator, len(group)) for operator, group in df_cores.groupby("oper")][:100]:
    print(item)

# Build Items and Submit to ScienceBase
Now that we have the basic information to start with from the two MapServer layers, we can assemble the particular data structure in current use for the NDC. This is based on the ScienceBase Item model as we've currently instantiated the NDC as collections of items in the ScienceBase Catalog. As we'll discuss later, this model has a number of limitations, but it is what we are using today, so I'm making the best use of it that I can.

All of the logic for building these items comes from the set of helper functions in crcwc_to_sb. It starts with the sb_item_from_crcwc function, which specifies the type of item and sends the item content (GeoJSON feature) from the MapServer retrieval we ran earlier. We send the "sample_type" variable to the functions as a convenience to switch between the configurations needed for each source schema, which are similar but have distinct differences. I started down the path of detecting the inbound schema, mappoing everything to common properties, and then acting accordingly, but that process would take a fair bit longer to work out and code up, so it is ugly code for the time being.

I made a few design choices here, essentially because ScienceBase is cranky about how many requests we can pass in short order. This would be a heck of a lot faster with multiprocessing, and we should be able to run it in parallel except for the limitations of working over HTTP with cranky APIs. So, I take some advantage of this need to slow things down and go ahead and assemble some related information into the items in real time.

The CRC database has some additional interesting content that is not expressed through the MapServer layers. This includes interval information on the depth ranges within a core, including geologic formation and age values, tables of thin sections that were taken from some samples, links to core photos, and links to downloadable files (variety of related information referred to as "analysis" files). I use the Beautifulsoup package to scrape this information from the web pages, which is the only way that it is currently expressed, and inject it into the prepared items.

I also used this opportunity to demonstrate something that we would likely provide as an information enhancement across the entire NDC rather than collection by collection. The Macrostrat folks have built a nicely usable API to all of the various datasets they have assembled from multiple sources. Their relatively simple and fast API route used for mobile applications provides a method for retrieving basic information derived from geologic maps for any given point location. I used this to add tags to the items for geologic formation, age, and rock types. At least for these collections, we will be able to examine the use of these tags as value-added facets in the data that come into play because we are able to link out to a related and relevant information source and bring data into our context.

I parse out information from these processes into identifiers, title, body, contacts, tags, and web links within the ScienceBase Items. I went ahead and shoved the full set of raw data structures from the MapServer item properties, web site scrape, and Macrostrat API into the body of the items. While this is ugly looking, it does add a bunch of content to the search index as items are added to the catalog. It does not allow for directed searching, categorizing, and faceting, but it does enhance the potential findability of the records. This also represents a possible strategy we might follow for the nextgen NDC - start every collection that wants to contribute by harvesting every bit of content that we possibly can, map anything we can figure out into a core set of commonly understood properties, shove everything else into an "anytext" area to make it available for search, and improve semantic mapping incrementally over time.

In [ ]:
%%time
for item in cores:
    crc_identifier = make_identifier(item)

    if sb.find_items({"filter0": f"itemIdentifier={crc_identifier}"})["total"] == 0:
        new_item = sb.create_item(sb_item_from_crcwc("core", item))
        print(new_item["link"]["url"])


https://www.sciencebase.gov/catalog/item/5d83c7d8e4b0c4f70d06e1d7
https://www.sciencebase.gov/catalog/item/5d83c7dae4b0c4f70d06e1d9
https://www.sciencebase.gov/catalog/item/5d83c7dbe4b0c4f70d06e1db
https://www.sciencebase.gov/catalog/item/5d83c7dce4b0c4f70d06e1dd
https://www.sciencebase.gov/catalog/item/5d83c7dde4b0c4f70d06e1df
https://www.sciencebase.gov/catalog/item/5d83c7dee4b0c4f70d06e1e1
https://www.sciencebase.gov/catalog/item/5d83c7e0e4b0c4f70d06e1e3
https://www.sciencebase.gov/catalog/item/5d83c7e1e4b0c4f70d06e1e5
https://www.sciencebase.gov/catalog/item/5d83c7e2e4b0c4f70d06e1e7
https://www.sciencebase.gov/catalog/item/5d83c7e3e4b0c4f70d06e1e9
https://www.sciencebase.gov/catalog/item/5d83c7e4e4b0c4f70d06e1eb
https://www.sciencebase.gov/catalog/item/5d83c7e5e4b0c4f70d06e1ed
https://www.sciencebase.gov/catalog/item/5d83c7e7e4b0c4f70d06e1ef
https://www.sciencebase.gov/catalog/item/5d83c7e8e4b0c4f70d06e1f1
https://www.sciencebase.gov/catalog/item/5d83c7eae4b0c4f70d06e1f3
https://ww

https://www.sciencebase.gov/catalog/item/5d83c87ce4b0c4f70d06e337
https://www.sciencebase.gov/catalog/item/5d83c87de4b0c4f70d06e339
https://www.sciencebase.gov/catalog/item/5d83c87ee4b0c4f70d06e33b
https://www.sciencebase.gov/catalog/item/5d83c880e4b0c4f70d06e33d
https://www.sciencebase.gov/catalog/item/5d83c881e4b0c4f70d06e33f
https://www.sciencebase.gov/catalog/item/5d83c883e4b0c4f70d06e341
https://www.sciencebase.gov/catalog/item/5d83c884e4b0c4f70d06e343
https://www.sciencebase.gov/catalog/item/5d83c885e4b0c4f70d06e345
https://www.sciencebase.gov/catalog/item/5d83c886e4b0c4f70d06e347
https://www.sciencebase.gov/catalog/item/5d83c888e4b0c4f70d06e349
https://www.sciencebase.gov/catalog/item/5d83c889e4b0c4f70d06e34b
https://www.sciencebase.gov/catalog/item/5d83c88be4b0c4f70d06e34d
https://www.sciencebase.gov/catalog/item/5d83c895e4b0c4f70d06e34f
https://www.sciencebase.gov/catalog/item/5d83c896e4b0c4f70d06e351
https://www.sciencebase.gov/catalog/item/5d83c897e4b0c4f70d06e353
https://ww

https://www.sciencebase.gov/catalog/item/5d83c92ce4b0c4f70d06e4ff
https://www.sciencebase.gov/catalog/item/5d83c92ee4b0c4f70d06e501
https://www.sciencebase.gov/catalog/item/5d83c92fe4b0c4f70d06e503
https://www.sciencebase.gov/catalog/item/5d83c931e4b0c4f70d06e505
https://www.sciencebase.gov/catalog/item/5d83c932e4b0c4f70d06e507
https://www.sciencebase.gov/catalog/item/5d83c933e4b0c4f70d06e509
https://www.sciencebase.gov/catalog/item/5d83c934e4b0c4f70d06e50b
https://www.sciencebase.gov/catalog/item/5d83c935e4b0c4f70d06e50d
https://www.sciencebase.gov/catalog/item/5d83c937e4b0c4f70d06e50f
https://www.sciencebase.gov/catalog/item/5d83c938e4b0c4f70d06e511
https://www.sciencebase.gov/catalog/item/5d83c93ae4b0c4f70d06e513
https://www.sciencebase.gov/catalog/item/5d83c93be4b0c4f70d06e515
https://www.sciencebase.gov/catalog/item/5d83c93de4b0c4f70d06e517
https://www.sciencebase.gov/catalog/item/5d83c93fe4b0c4f70d06e519
https://www.sciencebase.gov/catalog/item/5d83c940e4b0c4f70d06e51b
https://ww

https://www.sciencebase.gov/catalog/item/5d83c9d6e4b0c4f70d06e83f
https://www.sciencebase.gov/catalog/item/5d83c9d8e4b0c4f70d06e841
https://www.sciencebase.gov/catalog/item/5d83c9d9e4b0c4f70d06e843
https://www.sciencebase.gov/catalog/item/5d83c9dae4b0c4f70d06e845
https://www.sciencebase.gov/catalog/item/5d83c9dce4b0c4f70d06e847
https://www.sciencebase.gov/catalog/item/5d83c9dde4b0c4f70d06e849
https://www.sciencebase.gov/catalog/item/5d83c9dee4b0c4f70d06e84b
https://www.sciencebase.gov/catalog/item/5d83c9dfe4b0c4f70d06e84d
https://www.sciencebase.gov/catalog/item/5d83c9e0e4b0c4f70d06e84f
https://www.sciencebase.gov/catalog/item/5d83c9e2e4b0c4f70d06e851
https://www.sciencebase.gov/catalog/item/5d83c9e2e4b0c4f70d06e853
https://www.sciencebase.gov/catalog/item/5d83c9e4e4b0c4f70d06e855
https://www.sciencebase.gov/catalog/item/5d83c9e6e4b0c4f70d06e857
https://www.sciencebase.gov/catalog/item/5d83c9e7e4b0c4f70d06e859
https://www.sciencebase.gov/catalog/item/5d83c9e9e4b0c4f70d06e85b
https://ww

https://www.sciencebase.gov/catalog/item/5d83cadbe4b0c4f70d06e9f3
https://www.sciencebase.gov/catalog/item/5d83cadce4b0c4f70d06e9f5
https://www.sciencebase.gov/catalog/item/5d83cadde4b0c4f70d06e9f7
https://www.sciencebase.gov/catalog/item/5d83cadfe4b0c4f70d06e9f9
https://www.sciencebase.gov/catalog/item/5d83cae0e4b0c4f70d06e9fb
https://www.sciencebase.gov/catalog/item/5d83cae1e4b0c4f70d06e9fd
https://www.sciencebase.gov/catalog/item/5d83cae2e4b0c4f70d06e9ff
https://www.sciencebase.gov/catalog/item/5d83cae3e4b0c4f70d06ea01
https://www.sciencebase.gov/catalog/item/5d83cae5e4b0c4f70d06ea03
https://www.sciencebase.gov/catalog/item/5d83cae6e4b0c4f70d06ea05
https://www.sciencebase.gov/catalog/item/5d83cae7e4b0c4f70d06ea07
https://www.sciencebase.gov/catalog/item/5d83cae9e4b0c4f70d06ea09
https://www.sciencebase.gov/catalog/item/5d83caeae4b0c4f70d06ea0b
https://www.sciencebase.gov/catalog/item/5d83caebe4b0c4f70d06ea0d
https://www.sciencebase.gov/catalog/item/5d83caede4b0c4f70d06ea0f
https://ww

...

https://www.sciencebase.gov/catalog/item/5d83d031e4b0c4f70d06f1dc
https://www.sciencebase.gov/catalog/item/5d83d033e4b0c4f70d06f1de
https://www.sciencebase.gov/catalog/item/5d83d034e4b0c4f70d06f1e0
https://www.sciencebase.gov/catalog/item/5d83d038e4b0c4f70d06f1e2
https://www.sciencebase.gov/catalog/item/5d83d039e4b0c4f70d06f1e4
https://www.sciencebase.gov/catalog/item/5d83d03ae4b0c4f70d06f1e6
https://www.sciencebase.gov/catalog/item/5d83d03ce4b0c4f70d06f1e8
https://www.sciencebase.gov/catalog/item/5d83d03de4b0c4f70d06f1ea
https://www.sciencebase.gov/catalog/item/5d83d03ee4b0c4f70d06f1ec
https://www.sciencebase.gov/catalog/item/5d83d03fe4b0c4f70d06f1ee
https://www.sciencebase.gov/catalog/item/5d83d040e4b0c4f70d06f1f0
https://www.sciencebase.gov/catalog/item/5d83d042e4b0c4f70d06f1f2
https://www.sciencebase.gov/catalog/item/5d83d043e4b0c4f70d06f1f4
https://www.sciencebase.gov/catalog/item/5d83d044e4b0c4f70d06f1f6
https://www.sciencebase.gov/catalog/item/5d83d045e4b0c4f70d06f1f8
https://ww

https://www.sciencebase.gov/catalog/item/5d83d0d3e4b0c4f70d06f2e0
https://www.sciencebase.gov/catalog/item/5d83d0d5e4b0c4f70d06f2e2
https://www.sciencebase.gov/catalog/item/5d83d0d6e4b0c4f70d06f2e4
https://www.sciencebase.gov/catalog/item/5d83d0d7e4b0c4f70d06f2e6
https://www.sciencebase.gov/catalog/item/5d83d0d8e4b0c4f70d06f2e8
https://www.sciencebase.gov/catalog/item/5d83d0d9e4b0c4f70d06f2ea
https://www.sciencebase.gov/catalog/item/5d83d0dbe4b0c4f70d06f2ec
https://www.sciencebase.gov/catalog/item/5d83d0dce4b0c4f70d06f2ee
https://www.sciencebase.gov/catalog/item/5d83d0dde4b0c4f70d06f2f0
https://www.sciencebase.gov/catalog/item/5d83d0dfe4b0c4f70d06f2f2
https://www.sciencebase.gov/catalog/item/5d83d0e0e4b0c4f70d06f2f5
https://www.sciencebase.gov/catalog/item/5d83d0e1e4b0c4f70d06f2f7
https://www.sciencebase.gov/catalog/item/5d83d0e2e4b0c4f70d06f2f9
https://www.sciencebase.gov/catalog/item/5d83d0e3e4b0c4f70d06f2fb
https://www.sciencebase.gov/catalog/item/5d83d0e5e4b0c4f70d06f2fd
https://ww

https://www.sciencebase.gov/catalog/item/5d83d172e4b0c4f70d06f3e0
https://www.sciencebase.gov/catalog/item/5d83d173e4b0c4f70d06f3e2
https://www.sciencebase.gov/catalog/item/5d83d174e4b0c4f70d06f3e4
https://www.sciencebase.gov/catalog/item/5d83d175e4b0c4f70d06f3e6
https://www.sciencebase.gov/catalog/item/5d83d176e4b0c4f70d06f3e8
https://www.sciencebase.gov/catalog/item/5d83d178e4b0c4f70d06f3ea
https://www.sciencebase.gov/catalog/item/5d83d179e4b0c4f70d06f3ec
https://www.sciencebase.gov/catalog/item/5d83d17ae4b0c4f70d06f3ee
https://www.sciencebase.gov/catalog/item/5d83d17be4b0c4f70d06f3f0
https://www.sciencebase.gov/catalog/item/5d83d17ce4b0c4f70d06f3f2
https://www.sciencebase.gov/catalog/item/5d83d17de4b0c4f70d06f3f4
https://www.sciencebase.gov/catalog/item/5d83d17fe4b0c4f70d06f3f6
https://www.sciencebase.gov/catalog/item/5d83d180e4b0c4f70d06f3f8
https://www.sciencebase.gov/catalog/item/5d83d181e4b0c4f70d06f3fa
https://www.sciencebase.gov/catalog/item/5d83d183e4b0c4f70d06f3fc
https://ww

https://www.sciencebase.gov/catalog/item/5d83d215e4b0c4f70d06f4df
https://www.sciencebase.gov/catalog/item/5d83d217e4b0c4f70d06f4e1
https://www.sciencebase.gov/catalog/item/5d83d218e4b0c4f70d06f4e3
https://www.sciencebase.gov/catalog/item/5d83d219e4b0c4f70d06f4e5
https://www.sciencebase.gov/catalog/item/5d83d21ae4b0c4f70d06f4e7
https://www.sciencebase.gov/catalog/item/5d83d21be4b0c4f70d06f4e9
https://www.sciencebase.gov/catalog/item/5d83d21ce4b0c4f70d06f4ec
https://www.sciencebase.gov/catalog/item/5d83d21de4b0c4f70d06f4ee
https://www.sciencebase.gov/catalog/item/5d83d21fe4b0c4f70d06f4f0
https://www.sciencebase.gov/catalog/item/5d83d220e4b0c4f70d06f4f2
https://www.sciencebase.gov/catalog/item/5d83d221e4b0c4f70d06f4f4
https://www.sciencebase.gov/catalog/item/5d83d222e4b0c4f70d06f4f6
https://www.sciencebase.gov/catalog/item/5d83d223e4b0c4f70d06f4f8
https://www.sciencebase.gov/catalog/item/5d83d225e4b0c4f70d06f4fa
https://www.sciencebase.gov/catalog/item/5d83d226e4b0c4f70d06f4fc
https://ww

https://www.sciencebase.gov/catalog/item/5d83d2b3e4b0c4f70d06f5e1
https://www.sciencebase.gov/catalog/item/5d83d2b4e4b0c4f70d06f5e3
https://www.sciencebase.gov/catalog/item/5d83d2b5e4b0c4f70d06f5e5
https://www.sciencebase.gov/catalog/item/5d83d2b6e4b0c4f70d06f5e7
https://www.sciencebase.gov/catalog/item/5d83d2b7e4b0c4f70d06f5e9
https://www.sciencebase.gov/catalog/item/5d83d2b9e4b0c4f70d06f5eb
https://www.sciencebase.gov/catalog/item/5d83d2bae4b0c4f70d06f5ed
https://www.sciencebase.gov/catalog/item/5d83d2bbe4b0c4f70d06f5ef
https://www.sciencebase.gov/catalog/item/5d83d2bce4b0c4f70d06f5f1
https://www.sciencebase.gov/catalog/item/5d83d2bde4b0c4f70d06f5f3
https://www.sciencebase.gov/catalog/item/5d83d2bee4b0c4f70d06f5f5
https://www.sciencebase.gov/catalog/item/5d83d2bfe4b0c4f70d06f5f7
https://www.sciencebase.gov/catalog/item/5d83d2c0e4b0c4f70d06f5f9
https://www.sciencebase.gov/catalog/item/5d83d2c2e4b0c4f70d06f5fb
https://www.sciencebase.gov/catalog/item/5d83d2c3e4b0c4f70d06f5fd
https://ww

check on...
missing fields
length of tags